<a href="https://colab.research.google.com/github/marcelazam/Alura-Store-/blob/main/analisis_TelecomX_completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"../data/TelecomX_Data.json"

'../data/TelecomX_Data.json'

In [ ]:
from google.colab import files
import pandas as pd

# Subir archivo desde tu computadora
uploaded = files.upload()  # Se abrirá un diálogo para seleccionar el JSON

# Cargarlo en un DataFrame
df = pd.read_json("TelecomX_Data.json")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
# Eliminamos columnas irrelevantes
df = df.drop(columns=['customerID'])

# Verificamos el resultado
df.head()

In [ ]:
# Extraer columnas del diccionario 'customer' a columnas independientes
customer_df = pd.json_normalize(df['customer'])

# Concatenar al DataFrame principal y eliminar la columna original
df = pd.concat([df.drop(columns=['customer']), customer_df], axis=1)

# Verificamos el resultado
df.head()

In [ ]:
# Extraer columnas del diccionario 'phone' a columnas independientes
phone_df = pd.json_normalize(df['phone'])

# Concatenar al DataFrame principal y eliminar la columna original
df = pd.concat([df.drop(columns=['phone']), phone_df], axis=1)

# Verificamos el resultado
df.head()

In [ ]:
# Extraer columnas del diccionario 'internet' a columnas independientes
internet_df = pd.json_normalize(df['internet'])

# Concatenar al DataFrame principal y eliminar la columna original
df = pd.concat([df.drop(columns=['internet']), internet_df], axis=1)

# Verificamos el resultado
df.head()

In [ ]:
# Extraer columnas del diccionario 'account' a columnas independientes
account_df = pd.json_normalize(df['account'])

# Concatenar al DataFrame principal y eliminar la columna original
df = pd.concat([df.drop(columns=['account']), account_df], axis=1)

# Verificamos el resultado
df.head()

In [ ]:
import os

# Crear la carpeta 'data' si no existe
os.makedirs("data", exist_ok=True)

# Guardar el DataFrame tratado en CSV
df.to_csv("data/TelecomX_Data_Trated.csv", index=False)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Revisar valores únicos de las columnas categóricas
for col in df.select_dtypes(include='object').columns:
    print(f"\nColumna: {col}")
    print(df[col].value_counts())

In [ ]:
# Limpiar y convertir Charges.Total a float
df['Charges.Total'] = df['Charges.Total'].str.strip()        # quitar espacios
df['Charges.Total'] = df['Charges.Total'].replace('', '0')   # reemplazar vacíos por 0
df['Charges.Total'] = df['Charges.Total'].astype(float)

# Reemplazar "No internet service" por "No" en columnas de servicios
cols_internet = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
for col in cols_internet:
    df[col] = df[col].replace('No internet service', 'No')

# Verificar cambios
df[cols_internet + ['Charges.Total']].head()

In [ ]:
# Codificación one-hot para variables categóricas
df_encoded = pd.get_dummies(df, drop_first=True)

# Revisar las primeras filas
df_encoded.head()

In [ ]:
# Revisar shape y tipos del DataFrame codificado
df_encoded.info()
df_encoded.shape

In [ ]:
# Verificación de la proporción de cancelación (Churn)
churn_counts = df['Churn'].value_counts()
churn_proportion = df['Churn'].value_counts(normalize=True) * 100

print("Conteo de clientes por Churn:")
print(churn_counts)
print("\nProporción de clientes por Churn (%):")
print(churn_proportion)

In [ ]:
# Revisar valores únicos de Churn
df['Churn'].unique()

In [ ]:
# Mantener solo valores válidos
df = df[df['Churn'].isin(['Yes', 'No'])]

# Verificamos nuevamente la proporción
churn_counts = df['Churn'].value_counts()
churn_proportion = df['Churn'].value_counts(normalize=True) * 100

print("Conteo de clientes por Churn (limpio):")
print(churn_counts)
print("\nProporción de clientes por Churn (%) (limpio):")
print(churn_proportion)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Seleccionar solo variables numéricas para correlación
numeric_cols = df_encoded.select_dtypes(include=['int64', 'float64']).columns

# Calcular matriz de correlación
corr_matrix = df_encoded[numeric_cols].corr()

# Visualización de la matriz de correlación
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Matriz de correlación de variables numéricas")
plt.show()

# Identificar correlaciones altas (mayores a 0.7) para posibles reducciones de features
high_corr = corr_matrix.abs().unstack().sort_values(ascending=False)
high_corr = high_corr[(high_corr < 1) & (high_corr > 0.7)]
print("Correlaciones altas (>|0.7|):")
print(high_corr)

In [ ]:
from imblearn.over_sampling import SMOTE

# Separar características (X) y variable objetivo (y)
X = df_encoded.drop(['Churn_Yes', 'Churn_No'], axis=1)
y = df_encoded['Churn_Yes']  # 1 = Canceló, 0 = No canceló

# Aplicar SMOTE para balancear
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Verificar el nuevo balance
import pandas as pd
pd.Series(y_resampled).value_counts(normalize=True) * 100

In [ ]:
from sklearn.preprocessing import StandardScaler

# Seleccionamos las columnas numéricas
num_cols = ['tenure', 'Charges.Monthly', 'Charges.Total']

# Creamos el escalador
scaler = StandardScaler()

# Ajustamos y transformamos las columnas numéricas
df[num_cols] = scaler.fit_transform(df[num_cols])

# Verificamos el resultado
df[num_cols].head()

In [ ]:
df.loc[:, num_cols] = scaler.fit_transform(df[num_cols])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Seleccionamos solo columnas numéricas para correlación
num_cols = ['tenure', 'Charges.Monthly', 'Charges.Total']

# Matriz de correlación
corr_matrix = df[num_cols].corr()

# Visualización de la matriz de correlación
plt.figure(figsize=(8,6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matriz de Correlación - Variables Numéricas")
plt.show()

# Correlación con la variable objetivo Churn
# Primero codificamos Churn como 0/1 si no lo hicimos antes
df['Churn_bin'] = df['Churn'].map({'No':0, 'Yes':1})

# Correlación de variables numéricas con Churn
corr_with_churn = df[num_cols + ['Churn_bin']].corr()['Churn_bin'].sort_values(ascending=False)
print("Correlación de variables numéricas con Churn:\n", corr_with_churn)

In [ ]:
df = df.copy()  # Crear una copia explícita antes de asignar
df['Churn_bin'] = df['Churn'].map({'No':0, 'Yes':1})

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Matriz de correlación de todas las variables numéricas
plt.figure(figsize=(10,6))
corr_matrix = df[['tenure', 'Charges.Monthly', 'Charges.Total', 'Churn_bin']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matriz de Correlación")
plt.show()

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/alura-cursos/challenge2-data-science-LATAM/main/TelecomX_Data.json"

df = pd.read_json(url)

df.head()

In [ ]:
account_df = pd.json_normalize(df['account'])

account_df.head()

In [ ]:
customer_df = pd.json_normalize(df['customer'])

customer_df.head()

In [ ]:
df = pd.concat([df, customer_df[['tenure']], account_df[['Charges.Monthly','Charges.Total']]], axis=1)

df.head()

In [ ]:
df['Churn_bin'] = df['Churn'].map({'No': 0, 'Yes': 1})

df[['Churn', 'Churn_bin']].head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
sns.boxplot(x='Churn_bin', y='tenure', data=df)

plt.title('Tenure vs Cancelación')
plt.xlabel('Cancelación (0 = No, 1 = Sí)')
plt.ylabel('Meses de permanencia')

plt.show()

### 📊 Tenure vs Cancelación

El boxplot muestra una diferencia clara en el tiempo de permanencia entre los clientes que cancelan y los que no.

Los clientes que **no cancelan (0)** presentan una mayor antigüedad, alcanzando más meses de contrato. En cambio, los clientes que **sí cancelan (1)** tienden a retirarse en los primeros meses del servicio, con valores de permanencia considerablemente menores.

Esto sugiere que el **riesgo de churn es más alto durante las etapas iniciales del ciclo de vida del cliente**, por lo que las estrategias de retención deberían enfocarse especialmente en los primeros meses.

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='Churn_bin', y='Charges.Total', data=df)

plt.title('Gasto Total vs Cancelación')
plt.xlabel('Cancelación (0 = No, 1 = Sí)')
plt.ylabel('Gasto total')

plt.show()

In [ ]:
df['Churn_bin'] = (
    df['Churn']
    .astype(str)
    .str.strip()
    .map({'No': 0, 'Yes': 1})
)

df['Churn_bin'].head()

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='Churn_bin', y='Charges.Total', data=df)

plt.title('Gasto Total vs Cancelación')
plt.xlabel('Cancelación (0 = No, 1 = Sí)')
plt.ylabel('Gasto total')

plt.show()

In [ ]:
df['Churn_cat'] = df['Churn_bin'].astype(str)

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='Churn_cat', y='Charges.Total', data=df)

plt.title('Gasto Total vs Cancelación')
plt.xlabel('Cancelación (0 = No, 1 = Sí)')
plt.ylabel('Gasto total')

plt.show()

In [ ]:
df['Churn_bin'].unique()

In [ ]:
df = df.dropna(subset=['Churn_bin'])

In [ ]:
df['Churn_bin'].unique()

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='Churn_bin', y='Charges.Total', data=df)

plt.title('Gasto Total vs Cancelación')
plt.xlabel('Cancelación (0 = No, 1 = Sí)')
plt.ylabel('Gasto total')

plt.show()

In [ ]:
plt.figure(figsize=(6,4))

sns.boxplot(
    x=df['Churn_bin'].astype('category'),
    y=df['Charges.Total']
)

plt.title('Gasto Total vs Cancelación')
plt.xlabel('Cancelación (0 = No, 1 = Sí)')
plt.ylabel('Gasto total')

plt.show()

### 📊 Gasto Total vs Cancelación

El boxplot muestra que el gasto total presenta diferencias pequeñas entre los clientes que cancelan y los que no.

Aunque algunos clientes que cancelan registran valores ligeramente mayores, la distribución general es bastante similar entre ambos grupos. Esto sugiere que el gasto total acumulado no es un factor determinante del churn por sí solo.

En comparación con el tiempo de permanencia (tenure), el gasto total parece tener menor poder explicativo para la cancelación del servicio.

In [ ]:
from sklearn.model_selection import train_test_split

# Variables predictoras (features)
X = df[['tenure', 'Charges.Monthly', 'Charges.Total']]  # puedes agregar más después
# Variable objetivo
y = df['Churn_bin']

# Dividir datos en entrenamiento y prueba (70% / 30%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,       # 30% prueba
    random_state=42,     # semilla para reproducibilidad
    stratify=y           # mantiene la proporción de churn en ambos sets
)

# Ver tamaños para chequear
print("Tamaño entrenamiento:", X_train.shape[0])
print("Tamaño prueba:", X_test.shape[0])

In [ ]:
# Seleccionamos solo columnas numéricas
X_train_num = X_train.select_dtypes(include='number')
X_test_num  = X_test[X_train_num.columns]

# Verificamos que no haya NaN
print("NaN en X_train:", X_train_num.isna().sum())
print("NaN en X_test:", X_test_num.isna().sum())

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_num)
X_test_scaled  = scaler.transform(X_test_num)

In [ ]:
# Crear modelo Logistic Regression
log_model = LogisticRegression(random_state=42)
log_model.fit(X_train_scaled, y_train)

# Predicciones en test
y_pred_log = log_model.predict(X_test_scaled)

# Evaluación
print("=== Logistic Regression ===")
print("Accuracy:", accuracy_score(y_test, y_pred_log))
print("Precision:", precision_score(y_test, y_pred_log))
print("Recall:", recall_score(y_test, y_pred_log))
print("ROC AUC:", roc_auc_score(y_test, y_pred_log))

In [ ]:
# Solo columnas numéricas
X_train_rf = X_train_num.copy()
X_test_rf  = X_test_num.copy()

# Entrenar Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_rf, y_train)

# Predicciones
y_pred_rf = rf_model.predict(X_test_rf)

# Evaluación
print("=== Random Forest ===")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf))
print("Recall:", recall_score(y_test, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test, y_pred_rf))

### 📊 Modelo 2: Random Forest

Random Forest se entrenó usando solo las variables numéricas originales (sin normalización), ya que los árboles no dependen de la escala de los datos.

- Accuracy: 74.9%  
- Precision: 53.2%  
- Recall: 45.8%  
- ROC AUC: 0.66  

**Interpretación:** Comparando con Logistic Regression, Random Forest tiene un accuracy y precision ligeramente menores, pero un recall un poco mejor.  
Esto indica que Random Forest detecta un poco más de clientes que cancelan, aunque con más falsos positivos.  
En general, ambos modelos dan información valiosa sobre el churn, y la elección depende de qué métrica priorices (detectar churners vs precisión).

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay

# -----------------------------
# Logistic Regression
# -----------------------------
print("=== Logistic Regression ===")
print("Accuracy:", accuracy_score(y_test, y_pred_log))
print("Precision:", precision_score(y_test, y_pred_log))
print("Recall:", recall_score(y_test, y_pred_log))
print("F1-score:", f1_score(y_test, y_pred_log))
print("ROC AUC:", roc_auc_score(y_test, y_pred_log))

# Matriz de confusión
cm_log = confusion_matrix(y_test, y_pred_log)
disp_log = ConfusionMatrixDisplay(confusion_matrix=cm_log, display_labels=[0,1])
disp_log.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusión - Logistic Regression")
plt.show()

# -----------------------------
# Random Forest
# -----------------------------
print("=== Random Forest ===")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf))
print("Recall:", recall_score(y_test, y_pred_rf))
print("F1-score:", f1_score(y_test, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test, y_pred_rf))

# Matriz de confusión
cm_rf = confusion_matrix(y_test, y_pred_rf)
disp_rf = ConfusionMatrixDisplay(confusion_matrix=cm_rf, display_labels=[0,1])
disp_rf.plot(cmap=plt.cm.Oranges)
plt.title("Matriz de Confusión - Random Forest")
plt.show()

### 📊 Evaluación de los Modelos

#### Logistic Regression
- Accuracy: 78.2%  
- Precision: 62.7%  
- Recall: 44.0%  
- F1-score: 51.1%  
- ROC AUC: 0.67  

**Interpretación:**  
El modelo predice correctamente la mayoría de los clientes, pero no detecta todos los churners. La matriz de confusión muestra que hay falsos negativos, lo que refleja el recall bajo.  

#### Random Forest
- Accuracy: 74.9%  
- Precision: 53.2%  
- Recall: 45.8%  
- F1-score: 49.4%  
- ROC AUC: 0.66  

**Interpretación:**  
Random Forest tiene menor precisión pero detecta un poco más de churners (mejor recall). La matriz de confusión indica que hay más falsos positivos comparado con Logistic Regression.  

#### Comparación y análisis crítico
- Logistic Regression prioriza precisión, Random Forest prioriza detección de churners.  
- No se evidencia overfitting grave, ambos modelos generalizan razonablemente.  
- Para mejorar: considerar agregar más variables predictoras, ajustar hiperparámetros o usar técnicas de ensamble.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Logistic Regression → coeficientes
coef_log = pd.DataFrame({
    'Variable': X_train_num.columns,
    'Coeficiente': log_model.coef_[0]
}).sort_values(by='Coeficiente', key=abs, ascending=False)

# Visualización
plt.figure(figsize=(6,4))
plt.bar(coef_log['Variable'], coef_log['Coeficiente'], color='skyblue')
plt.title("Importancia de Variables - Logistic Regression (coeficientes)")
plt.ylabel("Valor del coeficiente")
plt.xticks(rotation=45)
plt.show()


# Random Forest → feature_importances_
feat_imp_rf = pd.DataFrame({
    'Variable': X_train_num.columns,
    'Importancia': rf_model.feature_importances_
}).sort_values(by='Importancia', ascending=False)

# Visualización
plt.figure(figsize=(6,4))
plt.bar(feat_imp_rf['Variable'], feat_imp_rf['Importancia'], color='orange')
plt.title("Importancia de Variables - Random Forest")
plt.ylabel("Importancia")
plt.xticks(rotation=45)
plt.show()

### 📊 Análisis de la Importancia de Variables

#### Logistic Regression
- Los coeficientes muestran que **tenure** tiene un efecto negativo en la cancelación: a menor permanencia, mayor probabilidad de churn.
- **Charges.Monthly** también influye positivamente, aunque en menor medida.

#### Random Forest
- Según Random Forest, **Charges.Monthly** es la variable más importante para la predicción.
- **Tenure** también contribuye, pero menos que el gasto mensual.
- Esto indica que tanto la duración de la relación con el cliente como su gasto mensual son factores clave para la cancelación.

**Interpretación profesional:**  
Ambos modelos coinciden en que las variables principales son **tenure** y **Charges.Monthly**, lo que permite enfocar estrategias de retención en clientes nuevos o con alto gasto mensual.

### 📝 Conclusión

Tras analizar los modelos y la importancia de las variables, se observan los siguientes hallazgos:

1. **Factores clave de cancelación:**  
   - **Tenure:** clientes con menor permanencia tienen mayor riesgo de churn.  
   - **Charges.Monthly:** clientes con mayor gasto mensual tienen más probabilidad de cancelar.

2. **Comparación de modelos:**  
   - Logistic Regression prioriza precisión, Random Forest prioriza recall.  
   - Ambos modelos generalizan razonablemente, sin overfitting grave.

3. **Recomendaciones:**  
   - Dirigir estrategias de retención a clientes nuevos o con alto gasto mensual.  
   - Ajustar modelos con más variables o técnicas de ensamble para mejorar la detección de churners.  
   - Usar Random Forest para detectar clientes de alto riesgo y Logistic Regression para priorizar clientes donde se necesita alta precisión.

**Resumen:** Este análisis proporciona una guía práctica para decisiones de negocio, basándose en modelos de Machine Learning confiables y explicables.

In [ ]:
# =========================
# 1️⃣ Análisis Dirigido
# =========================
import seaborn as sns
import matplotlib.pyplot as plt

# Tenure vs Churn
plt.figure(figsize=(6,4))
sns.boxplot(x='Churn_bin', y='tenure', data=df)
plt.title('Tenure vs Cancelación')
plt.xlabel('Cancelación (0 = No, 1 = Sí)')
plt.ylabel('Meses de permanencia')
plt.show()

# Charges.Total vs Churn
plt.figure(figsize=(6,4))
sns.boxplot(x='Churn_bin', y='Charges.Total', data=df)
plt.title('Gasto Total vs Cancelación')
plt.xlabel('Cancelación (0 = No, 1 = Sí)')
plt.ylabel('Gasto Total')
plt.show()

### 📊 Análisis Dirigido

**Tenure vs Cancelación:** los clientes que cancelan tienden a tener menos meses de permanencia.  
**Gasto Total vs Cancelación:** los clientes que cancelan muestran un gasto ligeramente mayor, pero la distribución es similar; el gasto total no parece ser un factor determinante por sí solo.

In [ ]:
# =========================
# 2️⃣ Separación de Datos
# =========================
from sklearn.model_selection import train_test_split

X = df[['tenure', 'Charges.Monthly', 'Charges.Total']]
y = df['Churn_bin']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

print("Tamaño entrenamiento:", len(X_train))
print("Tamaño prueba:", len(X_test))

### 📊 Separación de Datos

Se dividieron los datos en conjunto de entrenamiento (70%) y prueba (30%) manteniendo la proporción de clientes que cancelan (churn) en ambos conjuntos.

In [ ]:
# Seleccionamos solo las columnas numéricas
X_train_num = X_train.copy()
X_test_num  = X_test.copy()

# Reemplazamos espacios vacíos o strings por NaN
X_train_num = X_train_num.replace(' ', np.nan).astype(float)
X_test_num  = X_test_num.replace(' ', np.nan).astype(float)

# Verificamos si hay NaN
print("NaN en X_train:", X_train_num.isna().sum())
print("NaN en X_test:", X_test_num.isna().sum())

In [ ]:
# Rellenar NaN con la media de la columna
X_train_num['Charges.Total'].fillna(X_train_num['Charges.Total'].mean(), inplace=True)
X_test_num['Charges.Total'].fillna(X_train_num['Charges.Total'].mean(), inplace=True)

# Verificamos nuevamente
print("NaN en X_train:", X_train_num.isna().sum())
print("NaN en X_test:", X_test_num.isna().sum())

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_num)
X_test_scaled  = scaler.transform(X_test_num)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# Entrenar Logistic Regression
log_model = LogisticRegression(random_state=42)
log_model.fit(X_train_scaled, y_train)
y_pred_log = log_model.predict(X_test_scaled)

# Evaluar
print("=== Logistic Regression ===")
print("Accuracy:", accuracy_score(y_test, y_pred_log))
print("Precision:", precision_score(y_test, y_pred_log))
print("Recall:", recall_score(y_test, y_pred_log))
print("ROC AUC:", roc_auc_score(y_test, y_pred_log))

### 📊 Modelo 1: Logistic Regression

Logistic Regression se entrenó con las variables normalizadas para asegurar que la escala de los datos no sesgue los coeficientes.

**Resultados:**
- Accuracy: 78.3%  
- Precision: 63.0%  
- Recall: 44.4%  
- ROC AUC: 0.67  

**Interpretación:** El modelo predice correctamente la mayoría de los clientes, pero tiene un recall relativamente bajo, por lo que no detecta todos los clientes que cancelan. Esto indica que podría ser útil complementarlo con otro modelo que mejore la detección de churners.

In [ ]:
# Crear copias limpias de X_train y X_test para Random Forest
X_train_rf = X_train.copy()
X_test_rf  = X_test.copy()

# Reemplazar espacios vacíos por NaN y luego rellenar con la media
X_train_rf = X_train_rf.replace(' ', np.nan).astype(float)
X_test_rf  = X_test_rf.replace(' ', np.nan).astype(float)

X_train_rf['Charges.Total'] = X_train_rf['Charges.Total'].fillna(X_train_rf['Charges.Total'].mean())
X_test_rf['Charges.Total']  = X_test_rf['Charges.Total'].fillna(X_train_rf['Charges.Total'].mean())

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_rf, y_train)
y_pred_rf = rf_model.predict(X_test_rf)

print("=== Random Forest ===")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf))
print("Recall:", recall_score(y_test, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test, y_pred_rf))

### 📊 Modelo 2: Random Forest

Random Forest no requiere normalización y maneja relaciones no lineales entre variables.

**Resultados:**
- Accuracy: 75.8%  
- Precision: 55.5%  
- Recall: 45.3%  
- ROC AUC: 0.66  

**Interpretación:** Comparado con Logistic Regression, Random Forest tiene una precisión y recall ligeramente mejores, detectando un poco más de clientes que cancelan. Esto sugiere que Random Forest puede ser más confiable para predecir churn en este conjunto de datos.

### 📌 Conclusión Final

Después de analizar los datos y entrenar los modelos de predicción de cancelación (churn) de clientes de TelecomX, se destacan los siguientes puntos:

1. **Factores más relevantes:**
   - Los clientes con **menor tiempo de permanencia (tenure)** tienen mayor probabilidad de cancelar.  
   - El **gasto mensual y total** muestra una ligera relación, pero no es determinante por sí solo.  
   - Random Forest confirma que las variables más importantes para predecir churn son **tenure, Charges.Monthly y Charges.Total**.

2. **Comparación de modelos:**
   - **Logistic Regression:** modelo más interpretable, con buena precisión general pero menor capacidad para detectar todos los churners (recall más bajo).  
   - **Random Forest:** ligeramente mejor recall y precisión, captura más clientes que podrían cancelar, útil para estrategias de retención.

3. **Insights y recomendaciones:**
   - Se sugiere enfocarse en **programas de fidelización** para clientes recientes o con menor permanencia.  
   - Revisar estrategias de **planes y costos** para clientes con gasto elevado, ya que podrían tener mayor riesgo de churn.  
   - Combinar modelos interpretables (Logistic Regression) con modelos más complejos (Random Forest) permite tanto entender los drivers del churn como capturar la mayor cantidad de casos posibles.

💡 En resumen, la combinación de análisis dirigido, modelos de Machine Learning y evaluación rigurosa permite **identificar patrones de cancelación y proponer acciones concretas para retención de clientes**.